In [3]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
import pickle

In [4]:
movies= pd.read_csv('movies.csv')

In [5]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
credits = pd.read_csv('credits.csv')

In [7]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [8]:
movies= movies.merge(credits, on='title')

In [9]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
# After analysis, I think movie_id, title, overview, genre,keywords, cast and crew and important features. 

movies= movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [11]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [12]:
#merging overview, genres, keywords, cast and crew into single feature - tags.


In [13]:
# preprocessing
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [14]:
movies.dropna(inplace=True)

In [15]:
movies.duplicated().sum()

0

In [16]:
movies.iloc[0].genres


'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [17]:
def nameConcater(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [18]:
movies['genres'] = movies['genres'].apply(nameConcater)

In [19]:
movies['keywords']= movies['keywords'].apply(nameConcater)

In [20]:
# from cast, taking top N actors in the movie
def nameConcaterN(obj):
    L=[]
    counter = 0
    for i in ast.literal_eval(obj):
        if counter!= 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [21]:
movies['cast'] = movies['cast'].apply(nameConcaterN)

In [22]:
def fetchDirector(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [23]:
movies['crew'] = movies['crew'].apply(fetchDirector)

In [24]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [25]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]


In [26]:
# removing all spaces, for better searching through tags
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ","") for i in x])

In [27]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]


In [28]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [29]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [30]:
new_df = movies[['movie_id','title','tags']]

In [31]:
new_df.head(1)

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [32]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [33]:
# lowercase all tags
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

In [34]:
new_df['tags'][0]

'in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver jamescameron'

In [35]:
## Transformation of Tags (performing vectorization of tags)

cv= CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()
 

In [36]:
vectors.shape

(4806, 5000)

In [37]:
# stemming 
PS= PorterStemmer()


In [38]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(PS.stem(i))
    
    return " ".join(y)
        
    

In [39]:
new_df['tags']= new_df['tags'].apply(stem)

In [40]:
# Calculation of Cosine- distance between vectors
similarity = cosine_similarity(vectors)

In [41]:
sorted(list(enumerate(similarity[0])), reverse= True, key= lambda x: x[1])[1:11]

[(539, 0.26089696604360174),
 (1194, 0.2581988897471611),
 (507, 0.25302403842552984),
 (260, 0.25110592822973776),
 (1216, 0.24944382578492943),
 (582, 0.24397501823713333),
 (1920, 0.243599382882345),
 (3730, 0.23904572186687872),
 (1444, 0.2344036154692477),
 (74, 0.2267786838055363)]

In [42]:
# main recommender function

def recommend(movie):
    movie_index= new_df[new_df['title']== movie].index[0]
    distance= similarity[movie_index]
    movies_list = sorted(list(enumerate(distance)), reverse= True, key= lambda x: x[1])[1:11]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)



In [43]:
# Test it out

recommend('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman & Robin
Batman
Amidst the Devil's Wings
Batman v Superman: Dawn of Justice
Defendor
Mi America
Batman Returns


In [47]:
pickle.dump(new_df.to_dict(), open('movie_d.pkl', 'wb'))

In [48]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))
